In [ ]:
# Load the dataset from a dataset on the s3 bucket, or in this example the local file path
# from btb.tuning.tunable import Tunable
# from btb.tuning import GPTuner
import btb
from pyteller.core import Pyteller
from pyteller.data import load_data
import pandas as pd
from scipy import stats
import numpy as np
from pyteller.data import load_data
from sklearn.metrics import mean_absolute_error
from mlprimitives.datasets import Dataset
from mlblocks import MLPipeline,load_pipeline

current_data, input_data = load_data('taxi')
timestamp_col = 'timestamp'
target_signal = 'value'
embedding = 55
epochs = 200

pipeline = 'pyteller/pipelines/pyteller/LSTM/LSTM_offset.json'
pred_length = 10

dataset=Dataset('data',current_data,current_data,mean_absolute_error,'timeseries','forecast',shuffle=False)

def cross_validate(hyperparameters=None):
    scores = []
    for X_train, X_test, y_train, y_test in dataset.get_splits(3):
        global pipeline
        pipeline = MLPipeline(pipeline)

        if hyperparameters:
            pipeline.set_hyperparameters(hyperparameters)

        pyteller,all_output=DIRMO(X_train,X_test,pipeline)
        # Thes test data is the first prediciton set becuase it has the most predictions
        scores.append(pyteller.evaluate(test_data=all_output['actual'], forecast=all_output['forecast'],
                                        metrics=['sMAPE']).values[0][0])
        # del pipeline,pyteller

        # scores.append(taxi_dataset.score(y_test, y_pred))

    return np.mean(scores)

def DIRMO(X_train,X_test,pipeline):

    # s=pred_length
    # n = int(pred_length / s)  # number of models
    # for model in range(0, n):
    #     current_offset = model * s
    # hyperparameters = {
    #
    #     "pyteller.primitives.preprocessing.rolling_window_sequences#1": {
    #         "offset": current_offset,
    #         "target_size": s,
    #         "window_size": embedding,
    #
    #     },
    #     "pyteller.primitives.postprocessing.flatten#1": {
    #         "pred_length": s
    #     },
    #     "pyteller.primitives.estimators.Sequential_DIRMO#1": {
    #         "embedding": embedding,
    #         "epochs":epochs
    #     }
    # }

    pyteller = Pyteller(
        pipeline=pipeline,
        pred_length=5,
        offset=2,
        timestamp_col=timestamp_col,
        target_signal=target_signal
        # static_variables=None,
        # entity_col='station',
        # entities='8A0',
        # hyperparameters=hyperparameters
    )

    # Fit the data to the pipeline.
    train = pyteller.fit(X_train)
    # forecast and evaluate
    output = pyteller.forecast(data=X_test, postprocessing=False,
                               predictions_only=False)

        # del pyteller

    import pandas as pd


    return pyteller,output



pipeline = MLPipeline(load_pipeline(pipeline))
tunable_hyperparameters = pipeline.get_tunable_hyperparameters(flat=True)
#The last predictions (with the highest offset) is what you would do if lazy. Compare to this

from btb.tuning import Tunable
tunable = Tunable.from_dict(tunable_hyperparameters)

from btb.tuning import GPTuner
default_score = cross_validate()
# default_score
tuner = GPTuner(tunable)
defaults = tunable.get_defaults()
tuner.record(defaults, default_score)
best_score = default_score
best_score = 0
best_proposal = defaults

for iteration in range(1):
    print("scoring pipeline {}".format(iteration + 1))
    proposal = tuner.propose()

    score = cross_validate(proposal)

    tuner.record(proposal, score)

    if score < best_score:
        print("New best found: {}".format(score))
        best_score = score
        best_proposal = proposal

pipeline = MLPipeline(pipeline)
pipeline.set_hyperparameters(best_proposal)

pyteller,all_output = DIRMO(current_data, input_data,pipeline)

import pickle
# with open('forecast_MIMO_BTB_ele_10.pkl', 'wb') as f:
#     pickle.dump(combined, f)
#
# with open('actual_MIMO_BTB_ele_10.pkl', 'wb') as f:
#     pickle.dump(all_output, f)
# with open('best_prop_MIMO_ele_10.pkl', 'wb') as f:
#     pickle.dump(best_proposal, f)

# Thes test data is the first prediciton set becuase it has the most predictions
scores = pyteller.evaluate(test_data=all_output['actual'], forecast=all_output['forecast'],
                           metrics=['MAPE', 'sMAPE'])